## Importing all the required libraries

In [379]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.multiclass import OneVsRestClassifier

### Reading the training dataset to Pandas DataFrame

In [380]:
data = pd.read_csv('train.csv')
data.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


### Getting the target variables to Y variable

In [381]:
Y = data['Severity']
Y.shape

(10000,)

### Dropoing the irrelevent columns from training data

In [382]:
data = data.drop(columns=['Severity','Accident_ID','Adverse_Weather_Metric','Accident_Type_Code'],axis=1)
data.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Max_Elevation,Violations
0,49.223744,14,22,71.285324,0.272118,78.04,31335.476824,3
1,62.465753,10,27,72.288058,0.423939,84.54,26024.711057,2
2,63.059361,13,16,66.362808,0.322604,78.86,39269.053927,3
3,48.082192,11,9,74.703737,0.337029,81.79,42771.499200,1
4,26.484018,13,25,47.948952,0.541140,77.16,35509.228515,2


### creating the Label Encoder object which will encode the target severities to numerical form

In [383]:
label_encode = LabelEncoder()
y = label_encode.fit_transform(Y)

In [384]:
x_train,x_test,y_train,y_test = train_test_split(data,y,test_size=0.2)

In [385]:
from xgboost import XGBClassifier, XGBRFClassifier

In [386]:
xgb = XGBClassifier(n_estimators=500,learning_rate=0.1,max_depth=10,reg_lambda=0.1,importance_type='total_gain',
                   objective='multi:softmax')
xgbrf = XGBRFClassifier()

In [389]:
xgb.fit(data,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [390]:
predictions = xgb.predict(x_test)
acc = 100 * (f1_score(y_test,predictions,average='weighted'))
acc

100.0

In [391]:
test_data = pd.read_csv('test.csv')
accident_id = test_data['Accident_ID']
print(test_data.shape)
test_data = test_data.drop(columns=['Accident_ID','Adverse_Weather_Metric','Accident_Type_Code'],axis=1)
predictions = xgb.predict(test_data)
predictions = label_encode.inverse_transform(predictions)
result_df = pd.DataFrame({'Accident_ID':accident_id,'Severity':predictions})
result_df.to_csv('Prediction_xgb.csv',index=False)

(2500, 11)


### Accuracy